In [ ]:
Pkg.add("Jacobi")
Pkg.add("GaussQuadrature")

In [41]:
using Jacobi

function b_n(n::Int64)
    numerator = -n*(n+1)
    denominator = (n+2)*(n+3)
    return numerator/denominator
end

function M_in(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1, j+1] = 2/(2*i+1) + (b_n(i)*b_n(i))*(2/(2*i+5))
            elseif i==j+2
                fullMatrix[i+1, j+1] = b_n(j)*(2/(2*j+5))
            elseif i==j-2
                fullMatrix[i+1, j+1] = b_n(i)*(2/(2*j+5))
            end
        end
    end
    return fullMatrix
end

function B_in(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1,j+1]=2/(2*i+1)
            elseif i==j+2
                fullMatrix[i+1,j+1]=b_n(j)*2/(2*j+5)
            end
        end
    end
    return fullMatrix
end

function L_in(N::Function, size::Int64)
    fullArray = zeros(size,size)
    values = zgj(size, 0, 0)
    weights = wgj(values, 0, 0)
    for i=0:size-1
        for j=0:size-1
            sumTotal = 0
            for k=0:size-1
                dp_i = djacobi(values[k+1],i,0,0)+b_n(i)*djacobi(values[k+1],i+2,0,0)
                dp_j = djacobi(values[k+1],j,0,0)+b_n(j)*djacobi(values[k+1],j+2,0,0)
                newPiece = weights[k+1]*N(values[k+1])*dp_i*dp_j
                sumTotal = sumTotal + newPiece
            end
            fullArray[i+1,j+1] = sumTotal
        end
    end
    return fullArray
end

function u_g_rhs(u_bar::Function, size::Int64)
    fullArray = zeros(size)
    values = zgj(size, 0, 0)
    weights = wgj(values, 0, 0)
    for i =0:size-1
        sumTotal = 0
        for k=0:size-1
            p_n = jacobi(values[k+1], i,0,0)
            sumTotal = sumTotal + weights[k+1]*p_n*u_bar(values[k+1])
        end
        fullArray[i+1] = sumTotal
    end
    return fullArray
end

function u_galerkin(u_bar::Function, size::Int64)
    u_lhs = M_in(size)
    u_rhs = u_g_rhs(u_bar, size)
    return inv(u_lhs)*u_rhs
end
    


u_galerkin (generic function with 1 method)

In [45]:
using Jacobi

function N_constant(value::Float64)
    return 1
end

function z_linear(value::Float64)
    return value
end

#L_in(N_constant, 5)
#u_g_rhs(z_linear, 5)
u_galerkin(z_linear,5)

5-element Array{Float64,1}:
 1.38778e-17
 0.994996   
 0.0        
 0.150508   
 0.0        